In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    %pip install -U wandb transformers evaluate huggingface_hub accelerate
    colab = True
except Exception as e:
    print("Not running on Google Colab, skip this cell!")
    colab = False

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.0 MB/s et

In [2]:
import wandb

wandb.init(project="advanced_ai_imdb_dataset", name="albert-train-run")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import json

# Load path to pre-processed data in configs folder
config_file = "vlsp_colab" if colab else "vlsp_local"
with open(f"configs/{config_file}.json") as f:
    configs = json.load(f)

print(configs)

{'path': '/content/drive/MyDrive/HCMUS/Grad/Master/Học phần/HP1/Advanced AI/Project/Source/reimple'}


In [4]:
from datasets import load_dataset

source_path = configs["path"]
format_name = "format0"
# format_name = "format1"
# train_set = load_dataset(
#     "json", data_files=f"./vlsp_preprocessed/{format_name}/train.jsonl", split="train"
# )
# val_set = load_dataset(
#     "json", data_files=f"./vlsp_preprocessed/{format_name}/dev.jsonl", split="train"
# )
# test_set = load_dataset(
#     "json", data_files=f"./vlsp_preprocessed/{format_name}/test.jsonl", split="train"
# )
dataset = load_dataset(
    "json",
    data_files={
        "train": f"{source_path}/vlsp_preprocessed/{format_name}/train.jsonl",
        "val": f"{source_path}/vlsp_preprocessed/{format_name}/dev.jsonl",
        "test": f"{source_path}/vlsp_preprocessed/{format_name}/test.jsonl",
    },
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=4)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def preprocess_token(example):
    return tokenizer(
        example["sentence"], padding="max_length", truncation=True, return_tensors="pt"
    )

In [7]:
tokenized_dataset = dataset.map(preprocess_token, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/599 [00:00<?, ? examples/s]

Map:   0%|          | 0/1437 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
import evaluate

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

In [10]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)['accuracy'],
        "f1_macro": f1.compute(predictions=predictions, references=labels, average="macro")['f1'],
        "f1_micro": f1.compute(predictions=predictions, references=labels, average="micro")['f1'],
        "f1_weighted": f1.compute(predictions=predictions, references=labels, average="weighted")['f1'],
        "precision_macro": precision.compute(predictions=predictions, references=labels, average="macro")['precision'],
        "precision_micro": precision.compute(predictions=predictions, references=labels, average="micro")['precision'],
        "precision_weighted": precision.compute(predictions=predictions, references=labels, average="weighted")['precision'],
        "recall_macro": recall.compute(predictions=predictions, references=labels, average="macro")['recall'],
        "recall_micro": recall.compute(predictions=predictions, references=labels, average="micro")['recall'],
        "recall_weighted": recall.compute(predictions=predictions, references=labels, average="weighted")['recall'],
    }

In [11]:
id2label = {0: 'AFFILIATION', 1: 'PART – WHOLE', 2: 'LOCATED', 3: 'PERSONAL - SOCIAL'}
label2id = {value: key for key, value in id2label.items()}
print(label2id)

{'AFFILIATION': 0, 'PART – WHOLE': 1, 'LOCATED': 2, 'PERSONAL - SOCIAL': 3}


In [12]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="albert_imdb",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    eval_steps=500,
    save_total_limit=1,
    # report_to="wandb",
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [13]:
tokenized_dataset["train"][0]

{'sentence': '<location1/> <sep> <location2/> <sep> Trong ảnh : Nghệ_thuật tạo hoa_văn trên trang_phục truyền_thống của người Mông hoa , tại <location2/> , <location1/> .',
 'label': 2,
 'input_ids': [2,
  13,
  1,
  19032,
  8197,
  1,
  13,
  1,
  18,
  3492,
  1,
  13,
  1,
  19032,
  9298,
  1,
  13,
  1,
  18,
  3492,
  1,
  13,
  38,
  14271,
  40,
  252,
  13,
  45,
  13,
  2723,
  438,
  1,
  38,
  7325,
  38,
  14341,
  20538,
  1,
  2686,
  26407,
  13,
  38,
  13119,
  1,
  3971,
  6335,
  8600,
  8944,
  1,
  96,
  3279,
  13,
  22936,
  13,
  2723,
  13324,
  49,
  21028,
  20538,
  13,
  15,
  5466,
  13,
  1,
  19032,
  9298,
  1,
  13,
  15,
  13,
  1,
  19032,
  8197,
  1,
  13,
  9,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


In [ ]:
trainer.evaluate()

In [ ]:
wandb.finish()